In [1]:
import requests
from IPython.display import display

import json
import os
import numpy as np

import pandas as pd
from pandas import DataFrame, Series
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 6)

In [2]:

def flat_json(jsonData, parentKey=None):
    flat_dict = {}
    for key in jsonData:
        if type(jsonData[key]).__name__ == 'dict':
            if parentKey:
                temp = flat_json(jsonData[key], parentKey + "_" + key)
                flat_dict = {**flat_dict, **temp}          
            else:
                temp = flat_json(jsonData[key], key)
                flat_dict = {**flat_dict, **temp}
        else:
            if parentKey:
                flat_dict[parentKey + "_" + key] = jsonData[key]
            else:
                flat_dict[key] = jsonData[key]

    return flat_dict

def get_list_loans(id):
    jsonData = json.load(open("dataFinish\showPayAndCommRecord\showPayAndCommRecord_{0}.json".format(id), "r", encoding='utf-8'))
    
    resultContent = jsonData["resultContent"]
    if "listLoans" not in resultContent:
        df = DataFrame()
    else:
        df = DataFrame(resultContent["listLoans"])
        df["listingId"] = id

    return df

def read_show_borrower_info(id):
    jsonData = json.load(open("data\showBorrowerInfo\showBorrowerInfo_{0}.json".format(id), "r", encoding='utf-8'))
    result = flat_json(jsonData["resultContent"])
    if "overdueTyps" in result:
        result["overdueTyps"] = " ".join(result["overdueTyps"])
        
    if "userAuthsList" in result:
        for item in result["userAuthsList"]:
            if item["code"] == 1:
                result["userAuthsList_学历认证"] = True         
            elif item["code"] == 2:
                result["userAuthsList_身份证认证"] = True
            elif item["code"] == 3:
                result["userAuthsList_手机认证"] = True
            elif item["code"] == 4:
                result["userAuthsList_人行征信认证"] = True 
            else:
                print(item)
        
        del result["userAuthsList"]
#         print(result)
    result["listingId"] = id
    return result

def print_only_1_value_column_name(df):
    for column in df.columns.values:
        if df[column].value_counts().shape[0] == 1:
            print(column)

def print_columns(df):
    for column in df.columns.values:
        print(column)    


In [3]:
df_listing_info = pd.read_csv("dataFinish/915-922_B_6.csv", encoding="utf-8")
df_listing_info = df_listing_info.rename(columns={'resultContent_listing_listingId': 'listingId'})
df_listing_info.drop(["Unnamed: 0"], axis=1, inplace=True)
df_listing_info["listingId"] = df_listing_info.listingId.astype(int)
# df_listing_info.drop(["Unnamed: 0.1"], axis=1, inplace=True)
# df_listing_info.drop(["result"], axis=1, inplace=True)
# df_listing_info.drop(["resultContent_userInfo_picture"], axis=1, inplace=True)

print_only_1_value_column_name(df_listing_info)

# for column in ["resultContent_authenticated","resultContent_availableBalance","resultContent_bidAmount","resultContent_debgIng","resultContent_fullBidAndDebtDealing","resultContent_internalIp","resultContent_islimit","resultContent_listingBorrowerOrLender","resultContent_listing_ispay","resultContent_listing_source","resultContent_listing_vouch","resultContent_repaymentMethod"]:
#     df_listing_info.drop([column], axis=1, inplace=True)

for column in df_listing_info.columns.values:
    if column.startswith("resultContent_"):
        df_listing_info = df_listing_info.rename(columns={column: column.replace("resultContent_", "")})
        
display(df_listing_info.shape)
display(df_listing_info)

listing_creditCode
listing_months


(6512, 25)

,listingSaveBid,listing_amount,listing_auditingDate,listing_borrowerId,listing_creationDate,listing_creditCode,listing_currentRate,listing_description,listing_funding,listing_listDuration,listing_listEndDate,listing_listType,listingId,listing_months,listing_showRate,listing_statusId,listing_title,loanUse,totalBidUsers,userInfo_creationDate,userInfo_id,userInfo_lastLoginDate,userInfo_role,userInfo_userName,resultMessage
0,False,2000.0,2017-12-27 01:31:17,61941893.0,2017-12-27 01:11:36,B,22.0,手机app用户的第6次闪电借款,2000.0,20.0,2017-12-27 01:31:17,21.0,91500227,6.0,22.0,12.0,手机app用户的第6次闪电借款,NaN,20.0,2017-08-21 20:29:00,61941893.0,2018-06-27 14:32:49,8.0,pdu2****53123,NaN
1,False,1309.0,2017-12-27 01:33:21,66565857.0,2017-12-27 01:13:31,B,22.0,手机app用户的第2次闪电借款,1309.0,15.0,2017-12-27 01:33:21,21.0,91500312,6.0,22.0,12.0,手机app用户的第2次闪电借款,NaN,20.0,2017-10-06 12:52:43,66565857.0,2018-07-14 18:48:52,12.0,pdu4****53125,NaN
2,False,2477.0,2017-12-27 01:26:06,21535517.0,2017-12-27 01:23:48,B,22.0,手机app用户的第27次闪电借款,2477.0,20.0,2017-12-27 01:26:06,21.0,91500823,6.0,22.0,12.0,手机app用户的第27次闪电借款,NaN,33.0,2016-03-30 10:28:34,21535517.0,2018-06-25 20:54:57,12.0,pdu4****67257,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6509,True,1049.0,2018-01-01 12:45:35,59702177.0,2018-01-01 12:19:38,B,18.0,pdu7****02304第6次借款,1049.0,15.0,2018-01-01 12:45:35,0.0,92299564,6.0,18.0,12.0,pdu7****02304第6次借款,NaN,1.0,2017-07-30 15:42:57,59702177.0,2018-04-30 17:09:52,8.0,pdu7****02304,NaN
6510,True,1500.0,2018-01-01 12:45:45,54709130.0,2018-01-01 12:22:47,B,18.0,手机app用户的第7次闪电借款,1500.0,20.0,2018-01-01 12:45:45,21.0,92299946,6.0,18.0,12.0,手机app用户的第7次闪电借款,日常消费,1.0,2017-06-09 23:01:49,54709130.0,2018-07-20 13:01:48,12.0,pdu3****34622,NaN
6511,True,1573.0,2018-01-01 12:45:46,24101209.0,2018-01-01 12:22:57,B,18.0,手机app用户的第5次闪电借款,1573.0,20.0,2018-01-01 12:45:46,21.0,92299975,6.0,18.0,12.0,手机app用户的第5次闪电借款,NaN,1.0,2016-05-14 23:43:06,24101209.0,2018-07-23 11:04:30,8.0,pdu8****40578,NaN


In [4]:
def combine_showPayAndCommRecord_to_csv():
    mypath = "data/showPayAndCommRecord"
    onlyfiles = [f for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath, f))]
    print("finish")

    temp_list = [get_list_loans(f.split("_")[1].split(".")[0]) for f in onlyfiles]
    df_temp = pd.concat(temp_list)
    print('finish')

    df_temp.to_csv("dataFinish/showPayAndCommRecord_915-922_B_6.csv", encoding="utf-8")
    display(df_temp)

In [5]:
df_pay = pd.read_csv("dataFinish/showPayAndCommRecord_915-922_B_6.csv", encoding="utf-8")
df_pay.drop(["Unnamed: 0"], axis=1, inplace=True)
display(df_pay)

,agentId,days,dueDate,dueInterest,listingId,ouverDueDays,owingAmount,owingDun,owingInterest,owingOverdue,owingPrincipal,paymentDate,repayAmount,statusId
0,0,-2,2018-01-27 01:31:17,0.0,91500227,0,0.0,0.0,0.0,0.0,0.0,2018-01-25 00:18:03,355.04,1
1,0,-3,2018-02-27 01:31:17,0.0,91500227,0,0.0,0.0,0.0,0.0,0.0,2018-02-24 19:49:22,355.04,1
2,0,-2,2018-03-27 01:31:17,0.0,91500227,0,0.0,0.0,0.0,0.0,0.0,2018-03-25 00:00:32,355.02,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34947,0,0,2018-05-01 12:45:47,0.0,92299975,0,0.0,0.0,0.0,0.0,0.0,2018-05-01 10:26:51,276.10,1
34948,0,0,2018-06-01 12:45:47,0.0,92299975,0,0.0,0.0,0.0,0.0,0.0,2018-06-01 08:21:14,276.10,1
34949,0,0,2018-07-01 12:45:47,0.0,92299975,0,0.0,0.0,0.0,0.0,0.0,2018-07-01 08:22:24,276.10,1


In [6]:
display(df_listing_info[df_listing_info.listingId == 91502591])
display(df_pay[df_pay.listingId == 91502591])

,listingSaveBid,listing_amount,listing_auditingDate,listing_borrowerId,listing_creationDate,listing_creditCode,listing_currentRate,listing_description,listing_funding,listing_listDuration,listing_listEndDate,listing_listType,listingId,listing_months,listing_showRate,listing_statusId,listing_title,loanUse,totalBidUsers,userInfo_creationDate,userInfo_id,userInfo_lastLoginDate,userInfo_role,userInfo_userName,resultMessage
9,False,798.0,2017-12-27 02:19:25,66132978.0,2017-12-27 02:00:42,B,22.0,手机app用户的第5次闪电借款,798.0,20.0,2017-12-27 02:19:25,21.0,91502591,6.0,22.0,4.0,手机app用户的第5次闪电借款,NaN,10.0,2017-10-01 09:47:20,66132978.0,2018-06-12 13:38:51,8.0,pdu0****73383,NaN


,agentId,days,dueDate,dueInterest,listingId,ouverDueDays,owingAmount,owingDun,owingInterest,owingOverdue,owingPrincipal,paymentDate,repayAmount,statusId
54,0,-2,2018-01-27 02:19:29,0.00,91502591,0,0.00,0.00,0.00,0.00,0.00,2018-01-25 11:10:13,141.66,1
55,0,-3,2018-02-27 02:19:29,0.00,91502591,0,0.00,0.00,0.00,0.00,0.00,2018-02-24 20:52:56,141.66,1
56,0,120,2018-03-27 02:19:29,105.60,91502591,0,247.26,94.80,9.92,10.80,131.74,NaN,0.00,0
57,0,89,2018-04-27 02:19:29,79.21,91502591,0,220.87,71.20,7.51,8.01,134.15,NaN,0.00,0
58,0,59,2018-05-27 02:19:29,53.69,91502591,0,195.35,48.38,5.05,5.31,136.61,NaN,0.00,0
59,0,28,2018-06-27 02:19:29,25.76,91502591,0,167.42,23.24,2.56,2.52,139.10,NaN,0.00,0


In [7]:
# def calc_owing_principal_rate(df_pay, days):
#     df_pay_30 = df_pay[df_pay.days > days]
#     df_owingPrincipal_30 = df_pay_30.groupby("listingId").sum()["owingPrincipal"].to_frame()
#     df_owingPrincipal_30["listingId"] = df_owingPrincipal_30.index

#     df_listing_info_owingPrincipal_30 = pd.merge(df_listing_info, df_owingPrincipal_30, on="listingId", how="left")

#     column_name = "rate_owingPrincipal_" + str(days)
#     df_listing_info_owingPrincipal_30[column_name] = df_listing_info_owingPrincipal_30.owingPrincipal.fillna(0) / df_listing_info_owingPrincipal_30["listing_funding"]
#     df_listing_info_owingPrincipal_30_filter = df_listing_info_owingPrincipal_30[df_listing_info_owingPrincipal_30[column_name] != 0]
# #     display(df_listing_info_owingPrincipal_30_filter[column_name].describe())
#     return df_listing_info_owingPrincipal_30_filter;

# display(calc_owing_principal_rate(df_pay, 30))


In [8]:
mypath = "data/showBorrowerInfo"
onlyfiles = [f for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath, f))]
display(onlyfiles[:2])

temp_list = [read_show_borrower_info(f.split("_")[1].split(".")[0]) for f in onlyfiles]
df_showBorrowerInfo = DataFrame(temp_list)

# df_temp = pd.concat(temp_list)
df_showBorrowerInfo["listingId"] = df_showBorrowerInfo.listingId.astype(int)
# df_showBorrowerInfo["userAuthsList_人行征信认证"].fillna("N", inplace="True")
# df_showBorrowerInfo["userAuthsList_学历认证"].fillna("N", inplace="True")
# df_showBorrowerInfo["userAuthsList_手机认证"].fillna("N", inplace="True")
# df_showBorrowerInfo["userAuthsList_身份证认证"].fillna("N", inplace="True")
df_showBorrowerInfo.fillna("Unknow", inplace="True")

display(df_showBorrowerInfo)

['showBorrowerInfo_91500227.json', 'showBorrowerInfo_91500312.json']

,age,balAmount,educationInfo_educationDegree,educationInfo_graduate,educationInfo_studyStyle,gender,listingId,overdueStatus,overdueTyps,registerDateStr,repaymentSourceType,userAuthsList_人行征信认证,userAuthsList_学历认证,userAuthsList_手机认证,userAuthsList_身份证认证,workInfo
0,21,0.0,Unknow,Unknow,Unknow,女,91500227,0,Unknow,2017-08-21,经营收入,Unknow,Unknow,Unknow,Unknow,暂未提供
1,32,0.0,Unknow,Unknow,Unknow,女,91500312,0,Unknow,2017-10-06,薪资收入,Unknow,Unknow,Unknow,Unknow,暂未提供
2,40,0.0,Unknow,Unknow,Unknow,女,91500823,0,Unknow,2016-03-30,薪资收入,Unknow,Unknow,Unknow,Unknow,暂未提供
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6114,28,0.0,Unknow,Unknow,Unknow,女,92295914,0,Unknow,2017-06-03,,Unknow,Unknow,Unknow,Unknow,暂未提供
6115,31,0.0,Unknow,Unknow,Unknow,女,92297612,0,Unknow,2017-06-30,,Unknow,Unknow,Unknow,Unknow,工薪族
6116,21,0.0,Unknow,Unknow,Unknow,女,92298430,0,Unknow,2017-10-01,薪资收入,Unknow,Unknow,Unknow,Unknow,暂未提供


In [9]:
df_merge = pd.merge(df_listing_info, df_showBorrowerInfo, on="listingId")
display(df_merge)

,listingSaveBid,listing_amount,listing_auditingDate,listing_borrowerId,listing_creationDate,listing_creditCode,listing_currentRate,listing_description,listing_funding,listing_listDuration,listing_listEndDate,listing_listType,listingId,listing_months,listing_showRate,listing_statusId,listing_title,loanUse,totalBidUsers,userInfo_creationDate,userInfo_id,userInfo_lastLoginDate,userInfo_role,userInfo_userName,resultMessage,age,balAmount,educationInfo_educationDegree,educationInfo_graduate,educationInfo_studyStyle,gender,overdueStatus,overdueTyps,registerDateStr,repaymentSourceType,userAuthsList_人行征信认证,userAuthsList_学历认证,userAuthsList_手机认证,userAuthsList_身份证认证,workInfo
0,False,2000.0,2017-12-27 01:31:17,61941893.0,2017-12-27 01:11:36,B,22.0,手机app用户的第6次闪电借款,2000.0,20.0,2017-12-27 01:31:17,21.0,91500227,6.0,22.0,12.0,手机app用户的第6次闪电借款,NaN,20.0,2017-08-21 20:29:00,61941893.0,2018-06-27 14:32:49,8.0,pdu2****53123,NaN,21,0.0,Unknow,Unknow,Unknow,女,0,Unknow,2017-08-21,经营收入,Unknow,Unknow,Unknow,Unknow,暂未提供
1,False,1309.0,2017-12-27 01:33:21,66565857.0,2017-12-27 01:13:31,B,22.0,手机app用户的第2次闪电借款,1309.0,15.0,2017-12-27 01:33:21,21.0,91500312,6.0,22.0,12.0,手机app用户的第2次闪电借款,NaN,20.0,2017-10-06 12:52:43,66565857.0,2018-07-14 18:48:52,12.0,pdu4****53125,NaN,32,0.0,Unknow,Unknow,Unknow,女,0,Unknow,2017-10-06,薪资收入,Unknow,Unknow,Unknow,Unknow,暂未提供
2,False,2477.0,2017-12-27 01:26:06,21535517.0,2017-12-27 01:23:48,B,22.0,手机app用户的第27次闪电借款,2477.0,20.0,2017-12-27 01:26:06,21.0,91500823,6.0,22.0,12.0,手机app用户的第27次闪电借款,NaN,33.0,2016-03-30 10:28:34,21535517.0,2018-06-25 20:54:57,12.0,pdu4****67257,NaN,40,0.0,Unknow,Unknow,Unknow,女,0,Unknow,2016-03-30,薪资收入,Unknow,Unknow,Unknow,Unknow,暂未提供
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6114,True,682.0,2018-01-01 12:42:49,54063779.0,2018-01-01 11:51:32,B,18.0,手机app用户的第3次闪电借款,682.0,15.0,2018-01-01 12:42:49,21.0,92295914,6.0,18.0,4.0,手机app用户的第3次闪电借款,NaN,1.0,2017-06-03 09:59:56,54063779.0,2018-01-02 22:39:49,8.0,pdu6****20104,NaN,28,0.0,Unknow,Unknow,Unknow,女,0,Unknow,2017-06-03,,Unknow,Unknow,Unknow,Unknow,暂未提供
6115,True,2800.0,2018-01-01 12:44:25,56779675.0,2018-01-01 12:05:06,B,18.0,手机app用户的第9次闪电借款,2800.0,15.0,2018-01-01 12:44:25,21.0,92297612,6.0,18.0,4.0,手机app用户的第9次闪电借款,NaN,1.0,2017-06-30 22:59:06,56779675.0,2018-02-01 11:40:01,8.0,pdu8****44836,NaN,31,0.0,Unknow,Unknow,Unknow,女,0,Unknow,2017-06-30,,Unknow,Unknow,Unknow,Unknow,工薪族
6116,True,2000.0,2018-01-01 13:02:12,66174635.0,2018-01-01 12:09:53,B,18.0,手机app用户的第2次闪电借款,2000.0,20.0,2018-01-01 13:02:12,21.0,92298430,6.0,18.0,4.0,手机app用户的第2次闪电借款,旅游,1.0,2017-10-01 18:47:04,66174635.0,2018-03-07 08:53:56,8.0,pdu3****01885,NaN,21,0.0,Unknow,Unknow,Unknow,女,0,Unknow,2017-10-01,薪资收入,Unknow,Unknow,Unknow,Unknow,暂未提供


In [10]:
def calc_owing_principal_rate(df_pay, days, df_listing_info):
    df_pay_filter = df_pay[df_pay.days > days]
    df_owingPrincipal_30 = df_pay_filter.groupby("listingId").sum()["owingPrincipal"].to_frame()
    df_owingPrincipal_30["listingId"] = df_owingPrincipal_30.index

    df_listing_info_owingPrincipal_30 = pd.merge(df_listing_info, df_owingPrincipal_30, on="listingId", how="left")

    column_name = "rate_owingPrincipal_" + str(days)
    df_listing_info_owingPrincipal_30[column_name] = df_listing_info_owingPrincipal_30.owingPrincipal.fillna(0) / df_listing_info_owingPrincipal_30["listing_funding"]
#    df_listing_info_owingPrincipal_30_filter = df_listing_info_owingPrincipal_30[df_listing_info_owingPrincipal_30[column_name] != 0]
#     display(df_listing_info_owingPrincipal_30_filter[column_name].describe())
    return df_listing_info_owingPrincipal_30;

In [11]:
df_merge_all = calc_owing_principal_rate(df_pay, 30, df_merge)
display(df_merge_all)

,listingSaveBid,listing_amount,listing_auditingDate,listing_borrowerId,listing_creationDate,listing_creditCode,listing_currentRate,listing_description,listing_funding,listing_listDuration,listing_listEndDate,listing_listType,listingId,listing_months,listing_showRate,listing_statusId,listing_title,loanUse,totalBidUsers,userInfo_creationDate,userInfo_id,userInfo_lastLoginDate,userInfo_role,userInfo_userName,resultMessage,age,balAmount,educationInfo_educationDegree,educationInfo_graduate,educationInfo_studyStyle,gender,overdueStatus,overdueTyps,registerDateStr,repaymentSourceType,userAuthsList_人行征信认证,userAuthsList_学历认证,userAuthsList_手机认证,userAuthsList_身份证认证,workInfo,owingPrincipal,rate_owingPrincipal_30
0,False,2000.0,2017-12-27 01:31:17,61941893.0,2017-12-27 01:11:36,B,22.0,手机app用户的第6次闪电借款,2000.0,20.0,2017-12-27 01:31:17,21.0,91500227,6.0,22.0,12.0,手机app用户的第6次闪电借款,NaN,20.0,2017-08-21 20:29:00,61941893.0,2018-06-27 14:32:49,8.0,pdu2****53123,NaN,21,0.0,Unknow,Unknow,Unknow,女,0,Unknow,2017-08-21,经营收入,Unknow,Unknow,Unknow,Unknow,暂未提供,NaN,0.000000
1,False,1309.0,2017-12-27 01:33:21,66565857.0,2017-12-27 01:13:31,B,22.0,手机app用户的第2次闪电借款,1309.0,15.0,2017-12-27 01:33:21,21.0,91500312,6.0,22.0,12.0,手机app用户的第2次闪电借款,NaN,20.0,2017-10-06 12:52:43,66565857.0,2018-07-14 18:48:52,12.0,pdu4****53125,NaN,32,0.0,Unknow,Unknow,Unknow,女,0,Unknow,2017-10-06,薪资收入,Unknow,Unknow,Unknow,Unknow,暂未提供,NaN,0.000000
2,False,2477.0,2017-12-27 01:26:06,21535517.0,2017-12-27 01:23:48,B,22.0,手机app用户的第27次闪电借款,2477.0,20.0,2017-12-27 01:26:06,21.0,91500823,6.0,22.0,12.0,手机app用户的第27次闪电借款,NaN,33.0,2016-03-30 10:28:34,21535517.0,2018-06-25 20:54:57,12.0,pdu4****67257,NaN,40,0.0,Unknow,Unknow,Unknow,女,0,Unknow,2016-03-30,薪资收入,Unknow,Unknow,Unknow,Unknow,暂未提供,NaN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6114,True,682.0,2018-01-01 12:42:49,54063779.0,2018-01-01 11:51:32,B,18.0,手机app用户的第3次闪电借款,682.0,15.0,2018-01-01 12:42:49,21.0,92295914,6.0,18.0,4.0,手机app用户的第3次闪电借款,NaN,1.0,2017-06-03 09:59:56,54063779.0,2018-01-02 22:39:49,8.0,pdu6****20104,NaN,28,0.0,Unknow,Unknow,Unknow,女,0,Unknow,2017-06-03,,Unknow,Unknow,Unknow,Unknow,暂未提供,454.57,0.666525
6115,True,2800.0,2018-01-01 12:44:25,56779675.0,2018-01-01 12:05:06,B,18.0,手机app用户的第9次闪电借款,2800.0,15.0,2018-01-01 12:44:25,21.0,92297612,6.0,18.0,4.0,手机app用户的第9次闪电借款,NaN,1.0,2017-06-30 22:59:06,56779675.0,2018-02-01 11:40:01,8.0,pdu8****44836,NaN,31,0.0,Unknow,Unknow,Unknow,女,0,Unknow,2017-06-30,,Unknow,Unknow,Unknow,Unknow,工薪族,1866.35,0.666554
6116,True,2000.0,2018-01-01 13:02:12,66174635.0,2018-01-01 12:09:53,B,18.0,手机app用户的第2次闪电借款,2000.0,20.0,2018-01-01 13:02:12,21.0,92298430,6.0,18.0,4.0,手机app用户的第2次闪电借款,旅游,1.0,2017-10-01 18:47:04,66174635.0,2018-03-07 08:53:56,8.0,pdu3****01885,NaN,21,0.0,Unknow,Unknow,Unknow,女,0,Unknow,2017-10-01,薪资收入,Unknow,Unknow,Unknow,Unknow,暂未提供,1007.24,0.503620


In [12]:
def feature_engineer(df):
#     for column in df.columns.values:
    df_filter = df[df.rate_owingPrincipal_30 > 0]
    display(df_filter)
    show_nor = True
    for column in ["listing_currentRate", "listing_funding", "listing_listDuration"
                  , "listing_listType", "listing_statusId", "loanUse", "totalBidUsers",
                  "userInfo_role", "age","balAmount","educationInfo_educationDegree","educationInfo_graduate","educationInfo_studyStyle","gender","overdueStatus","overdueTyps","registerDateStr","repaymentSourceType","userAuthsList_人行征信认证","userAuthsList_学历认证","userAuthsList_手机认证","userAuthsList_身份证认证","workInfo"]:
#         print(df[column].value_counts())
        display(pd.concat([df[column].value_counts(show_nor), df_filter[column].value_counts(show_nor)], axis=1))
        print("")

        #print(column)


df_merge_all["loanUse"].fillna("Unknow", inplace="True")
feature_engineer(df_merge_all)

# print("df_pay")

# feature_engineer(df_pay_0)

,listingSaveBid,listing_amount,listing_auditingDate,listing_borrowerId,listing_creationDate,listing_creditCode,listing_currentRate,listing_description,listing_funding,listing_listDuration,listing_listEndDate,listing_listType,listingId,listing_months,listing_showRate,listing_statusId,listing_title,loanUse,totalBidUsers,userInfo_creationDate,userInfo_id,userInfo_lastLoginDate,userInfo_role,userInfo_userName,resultMessage,age,balAmount,educationInfo_educationDegree,educationInfo_graduate,educationInfo_studyStyle,gender,overdueStatus,overdueTyps,registerDateStr,repaymentSourceType,userAuthsList_人行征信认证,userAuthsList_学历认证,userAuthsList_手机认证,userAuthsList_身份证认证,workInfo,owingPrincipal,rate_owingPrincipal_30
9,False,798.0,2017-12-27 02:19:25,66132978.0,2017-12-27 02:00:42,B,22.0,手机app用户的第5次闪电借款,798.0,20.0,2017-12-27 02:19:25,21.0,91502591,6.0,22.0,4.0,手机app用户的第5次闪电借款,Unknow,10.0,2017-10-01 09:47:20,66132978.0,2018-06-12 13:38:51,8.0,pdu0****73383,NaN,18,0.0,Unknow,Unknow,Unknow,女,0,Unknow,2017-10-01,,Unknow,Unknow,Unknow,Unknow,暂未提供,402.50,0.504386
13,False,574.0,2017-12-27 02:49:04,35789742.0,2017-12-27 02:30:15,B,22.0,手机app用户的第8次闪电借款,574.0,15.0,2017-12-27 02:49:04,21.0,91503972,6.0,22.0,4.0,手机app用户的第8次闪电借款,Unknow,10.0,2016-10-28 04:10:52,35789742.0,2018-01-06 03:29:49,12.0,pdu1****00146,NaN,24,0.0,Unknow,Unknow,Unknow,男,0,Unknow,2016-10-28,,Unknow,Unknow,Unknow,Unknow,暂未提供,473.92,0.825645
15,False,4470.0,2017-12-27 03:01:20,24011772.0,2017-12-27 02:41:33,B,22.0,手机app用户的第13次闪电借款,4470.0,20.0,2017-12-27 03:01:20,21.0,91504330,6.0,22.0,4.0,手机app用户的第13次闪电借款,Unknow,54.0,2016-05-13 13:55:25,24011772.0,2018-06-05 20:19:25,8.0,pdu7****75572,NaN,27,0.0,Unknow,Unknow,Unknow,女,0,Unknow,2016-05-13,,Unknow,Unknow,Unknow,Unknow,暂未提供,3690.75,0.825671
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6114,True,682.0,2018-01-01 12:42:49,54063779.0,2018-01-01 11:51:32,B,18.0,手机app用户的第3次闪电借款,682.0,15.0,2018-01-01 12:42:49,21.0,92295914,6.0,18.0,4.0,手机app用户的第3次闪电借款,Unknow,1.0,2017-06-03 09:59:56,54063779.0,2018-01-02 22:39:49,8.0,pdu6****20104,NaN,28,0.0,Unknow,Unknow,Unknow,女,0,Unknow,2017-06-03,,Unknow,Unknow,Unknow,Unknow,暂未提供,454.57,0.666525
6115,True,2800.0,2018-01-01 12:44:25,56779675.0,2018-01-01 12:05:06,B,18.0,手机app用户的第9次闪电借款,2800.0,15.0,2018-01-01 12:44:25,21.0,92297612,6.0,18.0,4.0,手机app用户的第9次闪电借款,Unknow,1.0,2017-06-30 22:59:06,56779675.0,2018-02-01 11:40:01,8.0,pdu8****44836,NaN,31,0.0,Unknow,Unknow,Unknow,女,0,Unknow,2017-06-30,,Unknow,Unknow,Unknow,Unknow,工薪族,1866.35,0.666554
6116,True,2000.0,2018-01-01 13:02:12,66174635.0,2018-01-01 12:09:53,B,18.0,手机app用户的第2次闪电借款,2000.0,20.0,2018-01-01 13:02:12,21.0,92298430,6.0,18.0,4.0,手机app用户的第2次闪电借款,旅游,1.0,2017-10-01 18:47:04,66174635.0,2018-03-07 08:53:56,8.0,pdu3****01885,NaN,21,0.0,Unknow,Unknow,Unknow,女,0,Unknow,2017-10-01,薪资收入,Unknow,Unknow,Unknow,Unknow,暂未提供,1007.24,0.503620


,listing_currentRate,listing_currentRate
18.0,0.8205,0.821429
22.0,0.1795,0.178571


,listing_funding,listing_funding
0.0,0.004904,NaN
100.0,0.000163,NaN
233.0,0.000163,NaN
...,...,...
24700.0,0.000163,NaN
25100.0,0.000163,NaN
28600.0,0.000163,NaN


,listing_listDuration,listing_listDuration
20.0,0.642472,0.578571
15.0,0.357528,0.421429


,listing_listType,listing_listType
0.0,0.119830,0.157143
21.0,0.720124,0.707143
55.0,0.160046,0.135714


...

,userAuthsList_人行征信认证,userAuthsList_人行征信认证
Unknow,0.999673,1.0
True,0.000327,NaN


,userAuthsList_学历认证,userAuthsList_学历认证
Unknow,0.997384,1.0
True,0.002616,NaN


,userAuthsList_手机认证,userAuthsList_手机认证
Unknow,0.995096,1.0
True,0.004904,NaN


,userAuthsList_身份证认证,userAuthsList_身份证认证
Unknow,0.995096,1.0
True,0.004904,NaN


,workInfo,workInfo
公务员,0.010136,NaN
工薪族,0.059343,0.257143
暂未提供,0.690208,0.614286
私营业主,0.239823,0.128571
网店卖家,0.000490,NaN


In [13]:
pd.set_option('display.max_rows', 10)
cut_range = np.append(0, np.append(np.arange(4,13)*5, 100))
df_cut = df_merge_all.groupby(pd.cut(df_merge_all.age.astype(int), cut_range)).count()
display(df_merge_all.shape[0])
df_cut["rate"] = df_cut["listingSaveBid"] / df_merge_all.shape[0]
# display(df_cut)
display(df_cut[["owingPrincipal", "rate_owingPrincipal_30", "rate"]])

df_merge_all_filter = df_merge_all[df_merge_all.owingPrincipal > 0]
df_cut_filter = df_merge_all_filter.groupby(pd.cut(df_merge_all_filter.age.astype(int), cut_range)).count()
df_cut_filter["rate"] = df_cut_filter["listingSaveBid"] / df_merge_all_filter.shape[0]
# display(df_cut_filter)
display(df_cut_filter[["owingPrincipal", "rate_owingPrincipal_30", "rate"]])
pd.set_option('display.max_rows', 6)

6117

,owingPrincipal,rate_owingPrincipal_30,rate
age,,,
"(0, 20]",9,268,0.043812
"(20, 25]",60,2426,0.399869
"(25, 30]",59,1842,0.302272
"(30, 35]",19,887,0.145333
"(35, 40]",12,363,0.059506
"(40, 45]",5,172,0.028118
"(45, 50]",5,104,0.017002
"(50, 55]",1,21,0.003433
"(55, 60]",0,4,0.000654


,owingPrincipal,rate_owingPrincipal_30,rate
age,,,
"(0, 20]",4,4,0.028571
"(20, 25]",43,43,0.307143
"(25, 30]",52,52,0.371429
"(30, 35]",18,18,0.128571
"(35, 40]",12,12,0.085714
"(40, 45]",5,5,0.035714
"(45, 50]",5,5,0.035714
"(50, 55]",1,1,0.007143
"(55, 60]",0,0,0.000000


In [18]:
exceed_count =df_merge_all[df_merge_all["rate_owingPrincipal_30"] > 0].rate_owingPrincipal_30.count() 
print(exceed_count)
print(df_merge_all.shape[0])
print(exceed_count / df_merge_all.shape[0])

140
6117
0.0228870361288
